#### General Steps to Follow

1. Importing Packages
2. Defining x_train, x_test, y_train, y_test
3. Building and Training the Model
4. Model Evaluation

### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

#### Add the repository directory path to the Python path

In [1]:
import os
import sys

REPO_DIR_PATH = os.path.normpath(os.path.join(
    os.path.join(os.path.dirname(os.getcwd()))))

sys.path.append(REPO_DIR_PATH)

In [2]:
from src.train_model import ModelTrainer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, GRU, Dropout, BatchNormalization, TimeDistributed, Activation
from tensorflow.keras.optimizers import Adam

2024-07-22 19:18:28.193212: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 19:18:29.344514: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 2) Defining x_train, x_test, y_train, y_test

In [3]:
trainer = ModelTrainer()

In [4]:
data = trainer.load_data()

In [5]:
X_train, X_test, y_train, y_test = trainer.split_data(data)

In [6]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (8, 5511, 101)
Shape of y_train: (8, 1375, 1)
Shape of X_test: (2, 5511, 101)
Shape of y_test: (2, 1375, 1)


### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 3) Building and Training the Model

Our goal is to build a network that will ingest a spectrogram and output a signal when it detects the trigger word. This network will use 4 layers:
* A convolutional layer
* Two GRU layers
* A dense layer. 

Here is the architecture we will use.

<img src="../images/model.png" style="width:600px;height:600px;">

##### 1D convolutional layer
One key layer of this model is the 1D convolutional step. 
* It inputs the 5511 step spectrogram.  Each step is a vector of 101 units.
* It outputs a 1375 step output
* This output is further processed by multiple layers to get the final $T_y = 1375$ step output. 
* This 1D convolutional layer plays a role of extracting low-level features and then possibly generating an output of a smaller dimension. 
* Computationally, the 1-D conv layer also helps speed up the model because now the GRU  can process only 1375 timesteps rather than 5511 timesteps. 

##### GRU, dense and sigmoid
* The two GRU layers read the sequence of inputs from left to right.
* A dense plus sigmoid layer makes a prediction for $y^{\langle t \rangle}$. 
* Because $y$ is a binary value (0 or 1), we use a sigmoid output at the last layer to estimate the chance of the output being 1, corresponding to the user having just said "activate".

#### Unidirectional RNN
* We use a **unidirectional RNN** rather than a bidirectional RNN. 
* This is really important for trigger word detection, since we want to be able to detect the trigger word almost immediately after it is said. 
* If we used a bidirectional RNN, we would have to wait for the whole 10sec of audio to be recorded before we could tell if "activate" was said in the first second of the audio clip.  

In [14]:
input_shape = (X_train.shape[1], X_train.shape[2])
input_shape

(5511, 101)

In [15]:
model = Sequential(
    [
        Conv1D(filters=196, kernel_size=15, strides=4, input_shape = input_shape),
        BatchNormalization(),
        Activation('relu'),
        Dropout(rate=0.8),
        GRU(128, return_sequences=True),
        Dropout(rate=0.8),
        BatchNormalization(),                          
        GRU(128, return_sequences=True),
        Dropout(rate=0.8),    
        BatchNormalization(),
        Dropout(rate=0.8),                                
        TimeDistributed(Dense(1, activation='sigmoid')) # Applies a dense (fully connected) layer to each time step independently.
    ]
)

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-07-22 15:53:56.168117: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 15:53:56.315513: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platfor

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1375, 196)      │       297,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1375, 196)      │           784 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1375, 196)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1375, 196)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 1375, 128)      │       125,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1375, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1375, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 1375, 128)      │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1375, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1375, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1375, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 1375, 1)        │           129 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 523,329 (2.00 MB)

 Trainable params: 522,425 (1.99 MB)

 Non-trainable params: 904 (3.53 KB)

In [17]:
opt = Adam(learning_rate=1e-6, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [18]:
model.fit(X_train, y_train, epochs=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 45s 45s/step - accuracy: 0.4975 - loss: 1.1802


### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 4) Model Evaluation

##### Evaluate on Training Data

In [28]:
model.evaluate(X_train, y_train)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.0936 - loss: 0.7073


[0.7072852253913879, 0.09363636374473572]

##### Evaluate on Testing Data

In [30]:
model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.1087 - loss: 0.7046


[0.7045834064483643, 0.10872727632522583]